# Ejercicios
Cada línea del archivo movies.csv representa a un actor que actuó en una película.\
Si una película tiene diez actores, habrá diez filas para esa película en particular.

In [28]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import desc, col
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import udf
from pyspark.sql.window import Window
from pyspark.sql.functions import rank
from pyspark.sql.functions import count_distinct

path = 'files/movies/'

In [2]:
spark = SparkSession.builder.getOrCreate()

24/01/10 10:56:23 WARN Utils: Your hostname, luis-Nitro-AN515-52 resolves to a loopback address: 127.0.1.1; using 192.168.1.17 instead (on interface enp7s0f1)
24/01/10 10:56:23 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/01/10 10:56:25 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [11]:
# Lectura de los archivos
movies = spark.read.option('header', 'true').option('inferSchema','true').option('delimiter','|').csv(path+'movies.csv')
ratings = spark.read.option('header', 'true').option('inferSchema','true').option('delimiter','|').csv(path+'movie_ratings.csv')

In [12]:
movies.show(5)

+-----------------+-------------+----+
|            actor|     pelicula| año|
+-----------------+-------------+----+
|McClure, Marc (I)|Freaky Friday|2003|
|McClure, Marc (I)| Coach Carter|2005|
|McClure, Marc (I)|  Superman II|1980|
|McClure, Marc (I)|    Apollo 13|1995|
|McClure, Marc (I)|     Superman|1978|
+-----------------+-------------+----+
only showing top 5 rows



In [13]:
ratings.show(5)

+----------+--------------------+----+
|valoracion|            pelicula| año|
+----------+--------------------+----+
|    1.6339|'Crocodile' Dunde...|1988|
|    7.6177|                  10|1979|
|    1.2864|10 Things I Hate ...|1999|
|    0.3243|           10,000 BC|2008|
|    0.3376|      101 Dalmatians|1996|
+----------+--------------------+----+
only showing top 5 rows



1. Calcule la cantidad de películas en las que participó cada actor. La salida debe tener dos columnas: actor y conteo. La salida debe ordenarse por el conteo en orden descendente.

In [32]:
movies.groupBy('actor')\
    .count()\
    .orderBy(desc('count'))\
    .show()

+-------------------+-----+
|              actor|count|
+-------------------+-----+
|   Tatasciore, Fred|   38|
|      Welker, Frank|   38|
| Jackson, Samuel L.|   32|
|      Harnell, Jess|   31|
|        Damon, Matt|   27|
|      Willis, Bruce|   27|
|  Cummings, Jim (I)|   26|
|         Hanks, Tom|   25|
|   Lynn, Sherry (I)|   25|
|    McGowan, Mickie|   25|
|    Bergen, Bob (I)|   25|
|      Proctor, Phil|   24|
|   Wilson, Owen (I)|   23|
|        Cruise, Tom|   23|
|         Pitt, Brad|   23|
|Freeman, Morgan (I)|   22|
|Williams, Robin (I)|   22|
|       Depp, Johnny|   22|
|     Morrison, Rana|   22|
|      Diaz, Cameron|   21|
+-------------------+-----+
only showing top 20 rows



2.     Calcule la cantidad de películas producidas cada año. La salida debe tener tres columnas: año, siglo al que pertenece el año y conteo. La salida debe ordenarse por el conteo en orden descendente.




In [22]:
# Funcion siglo
@udf(returnType=IntegerType())
def siglo(s):
    return 1+(s//100)

In [34]:
movies.groupBy('año').agg(
    count_distinct('pelicula').alias('conteo'),
    siglo(col('año')).alias('siglo')
    )\
    .orderBy(desc('count'))\
    .show()

+----+-----+-----+
| año|count|siglo|
+----+-----+-----+
|2006|   86|   20|
|2004|   86|   20|
|2005|   85|   20|
|2011|   85|   20|
|2008|   82|   20|
|2002|   81|   20|
|2000|   77|   20|
|2003|   76|   20|
|2010|   76|   20|
|2007|   75|   20|
|2001|   71|   20|
|2009|   68|   20|
|1999|   67|   19|
|1997|   66|   19|
|1998|   59|   19|
|1996|   42|   19|
|2012|   32|   20|
|1995|   25|   19|
|1994|   16|   19|
|1986|   16|   19|
+----+-----+-----+
only showing top 20 rows



3. Obtenga la película con la calificación más alta por año. La salida debe tener solo una película por año y debe contener tres columnas: año, título de la película y valoración.

In [27]:
windowSpec = Window.partitionBy('año').orderBy(desc('valoracion'))

In [31]:
ratings.withColumn('rank', rank().over(windowSpec))\
    .filter(col('rank')==1)\
    .select(col('año'),col('pelicula'),col('valoracion'))\
    .show()

+----+--------------------+----------+
| año|            pelicula|valoracion|
+----+--------------------+----------+
|1937|Snow White and th...|    2.2207|
|1939|    The Wizard of Oz|    7.9215|
|1940|           Pinocchio|    7.8557|
|1942|               Bambi|    1.5053|
|1946|   Song of the South|     7.602|
|1950|          Cinderella|    9.4226|
|1953|           Peter Pan|    5.4756|
|1954|         Rear Window|   10.7625|
|1955|  Lady and the Tramp|    5.1258|
|1956|Around the World ...|   14.0607|
|1959|     Sleeping Beauty|    6.3919|
|1960|              Psycho|   10.6375|
|1961|One Hundred and O...|    0.6726|
|1962|     The Longest Day|   12.8866|
|1963|It's a Mad Mad Ma...|     6.626|
|1964|        My Fair Lady|     7.587|
|1965|      Doctor Zhivago|    4.9304|
|1966|Who's Afraid of V...|   11.1111|
|1967|     The Dirty Dozen|    13.388|
|1968|        The Love Bug|   13.4383|
+----+--------------------+----------+
only showing top 20 rows



In [ ]:
spark.sparkContext.stop()